In [ ]:
# from docling.document_converter import DocumentConverter

# # source = "Phillips  Cohen - MSA - 00411904.0 - fully signed.pdf"
# source = "Avid Technology Inc - Global Terms and Conditions_EXECUTED_05.18.2.pdf"
# converter = DocumentConverter()
# result = converter.convert(source)

# structured_text = result.document.export_to_markdown()

def chunk_markdown_by_heading(structured_text):
    """
    Chunks markdown text based on H2 headings (##).
    Each chunk starts with an H2 heading and includes all content until the next H2.
    """
    lines = structured_text.strip().split('\n')
    chunks = []
    current_chunk_lines = []
    chunk_number = 1

    for line in lines:
        # Check for H2 heading, but not H3 (to avoid sub-sub-sections breaking main chunks)
        if line.strip().startswith("##") and not line.strip().startswith("###"):
            if current_chunk_lines:
                # If there's content in the current chunk, save it
                content = '\n'.join(current_chunk_lines).strip()
                chunks.append((f"chunk {chunk_number}", content))
                chunk_number += 1
                current_chunk_lines = [] # Reset for the new chunk
        current_chunk_lines.append(line)

    # Add the last chunk if it exists
    if current_chunk_lines:
        content = '\n'.join(current_chunk_lines).strip()
        chunks.append((f"chunk {chunk_number}", content))
    return chunks

chunk_markdown_by_heading("Global_Gourmet_Traders_4_Page_Report.pdf")

[('chunk 1', 'Global_Gourmet_Traders_4_Page_Report.pdf')]

In [83]:
print(structured_text)

&lt;unknown&gt;

<!-- image -->

## GLOBAL TERMS AND CONDITIONS

- Structure These Global Terms and Conditions ("Global Terms and Conditions") are incorporated by reference into each country-specific Master Services Agreement entered into by Customer and the applicable Digital Realty entity (each; a 'Country MSA") . The terms "Customer" and "Digital Realty" are defined in the applicable Country MSA Customer and Digital Realty may enter into orders (whether executed by manual or electronic   signature or via Digital   Realty's online   ordering platform; each an "Order"), each of which incorporates the terms of the applicable Country MSA and constitutes separate and distinct contract for data center-related services andlor products to be provided by Digital Realty to Customer ("Services"), as more particularly set forth on each Order. An Order; together with MSA and these Global Terms and Conditions; is referred to as the "Agreement. Services may not be ordered, activated, or used by Cu

### pymupdf4llm

In [ ]:
# import pymupdf4llm
# import pathlib

# # Specify the input PDF file path
# pdf_path = "KakaoBank - Global MSA and GTC_EXECUTED_08.04.23.pdf"
# # Convert the PDF to Markdown text

# md_text = pymupdf4llm.to_markdown(pdf_path)

# # Optionally, save the Markdown text to a file
# # output_md_path = "output.md"
# # pathlib.Path(output_md_path).write_bytes(md_text.encode("utf-8"))
# # print(f"PDF converted to Markdown and saved to {output_md_path}")

In [ ]:
# print(structured_text)

&lt;unknown&gt;

<!-- image -->

## GLOBAL TERMS AND CONDITIONS

- Structure These Global Terms and Conditions ("Global Terms and Conditions") are incorporated by reference into each country-specific Master Services Agreement entered into by Customer and the applicable Digital Realty entity (each; a 'Country MSA") . The terms "Customer" and "Digital Realty" are defined in the applicable Country MSA Customer and Digital Realty may enter into orders (whether executed by manual or electronic   signature or via Digital   Realty's online   ordering platform; each an "Order"), each of which incorporates the terms of the applicable Country MSA and constitutes separate and distinct contract for data center-related services andlor products to be provided by Digital Realty to Customer ("Services"), as more particularly set forth on each Order. An Order; together with MSA and these Global Terms and Conditions; is referred to as the "Agreement. Services may not be ordered, activated, or used by Cu

In [ ]:
# type(structured_text)

str

In [47]:
lines = structured_text.strip().split('\n')

In [51]:
print("Number of lines:", (lines))

Number of lines: ['&lt;unknown&gt;', '', '<!-- image -->', '', '## GLOBAL TERMS AND CONDITIONS', '', '- Structure These Global Terms and Conditions ("Global Terms and Conditions") are incorporated by reference into each country-specific Master Services Agreement entered into by Customer and the applicable Digital Realty entity (each; a \'Country MSA") . The terms "Customer" and "Digital Realty" are defined in the applicable Country MSA Customer and Digital Realty may enter into orders (whether executed by manual or electronic   signature or via Digital   Realty\'s online   ordering platform; each an "Order"), each of which incorporates the terms of the applicable Country MSA and constitutes separate and distinct contract for data center-related services andlor products to be provided by Digital Realty to Customer ("Services"), as more particularly set forth on each Order. An Order; together with MSA and these Global Terms and Conditions; is referred to as the "Agreement. Services may n

In [49]:
lines[0]

'&lt;unknown&gt;'

In [ ]:
chunks = []            # List to hold each chunk
current_chunk = []     # Temporary list to hold lines of a chunk
chunk_number = 1       # To number the chunks


# Step 2: Loop through lines and detect headings
for line in lines:
        if line.strip().startswith("##") and not line.strip().startswith("###"): 
              # If heading
            if current_chunk:  # Save previous chunk before starting new
                content = '\n'.join(current_chunk).strip()
                chunks.append((f"chunk {chunk_number}", content))
                chunk_number += 1
                current_chunk = []
        current_chunk.append(line)

# Step 3: Add the last chunk
if current_chunk:
    content = '\n'.join(current_chunk).strip()
    chunks.append((f"chunk {chunk_number}", content))


## Tokenizer

In [68]:
from typing import List
import nltk
from transformers import AutoTokenizer

# nltk.download('punkt')  # Uncomment if running for the first time

def chunk_text_for_embedding(
    chunk_texts: List[str],
    model_name: str = "BAAI/bge-base-en-v1.5",
    max_tokens: int = 512,
    buffer_tokens: int = 20
) -> List[str]:
    """
    Splits input text chunks into smaller chunks that fit within the embedding model's token limit.

    Args:
        chunk_texts: List of input text chunks.
        model_name: Name of the HuggingFace model to load the tokenizer.
        max_tokens: Max tokens allowed per chunk.
        buffer_tokens: Safety margin for tokenizer variations.

    Returns:
        List of text chunks, each within token limits.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    all_chunks = []

    for i, text in enumerate(chunk_texts):
        sentences = nltk.sent_tokenize(text)
        # print(f"Number of sentences in chunk {i+1}: {len(sentences)}")

        current_chunk = []
        current_tokens = 0

        for sentence in sentences:
            token_count = len(tokenizer.encode(sentence, add_special_tokens=False))
            if current_tokens + token_count <= max_tokens - buffer_tokens:
                current_chunk.append(sentence)
                current_tokens += token_count
            else:
                all_chunks.append(" ".join(current_chunk))
                current_chunk = [sentence]
                current_tokens = token_count

        # Append last chunk for this chunk_text
        if current_chunk:
            all_chunks.append(" ".join(current_chunk))

    return all_chunks

# Usage example:
# chunk_texts = [text for _, text in chunks]
# token_chunks = chunk_text_for_embedding(chunk_texts)

In [69]:
# chunk_text_for_embedding(chunks)
chunk_text_for_embedding([text for _, text in chunks])

Token indices sequence length is longer than the specified maximum sequence length for this model (537 > 512). Running this sequence through the model will result in indexing errors


['&lt;unknown&gt;\n\n<!-- image -->',
 '## GLOBAL TERMS AND CONDITIONS\n\n- Structure These Global Terms and Conditions ("Global Terms and Conditions") are incorporated by reference into each country-specific Master Services Agreement entered into by Customer and the applicable Digital Realty entity (each; a \'Country MSA") . The terms "Customer" and "Digital Realty" are defined in the applicable Country MSA Customer and Digital Realty may enter into orders (whether executed by manual or electronic   signature or via Digital   Realty\'s online   ordering platform; each an "Order"), each of which incorporates the terms of the applicable Country MSA and constitutes separate and distinct contract for data center-related services andlor products to be provided by Digital Realty to Customer ("Services"), as more particularly set forth on each Order. An Order; together with MSA and these Global Terms and Conditions; is referred to as the "Agreement. Services may not be ordered, activated, or

In [12]:
from llama_index.core.schema import Document  # Add this import

documents = [Document(text=text) for _, text in chunks]

In [14]:
# Separate out chunk_texts and metadata
# documents = [text for _, text in chunks]

from uuid import uuid4

metadatas = [
    {"chunk_id": chunk_id, "source": "delivery_report.md"}  # You can add more fields here
    for chunk_id, _ in chunks ]

ids = [str(uuid4()) for _ in chunks]


metadatas

[{'chunk_id': 'chunk 1', 'source': 'delivery_report.md'},
 {'chunk_id': 'chunk 2', 'source': 'delivery_report.md'},
 {'chunk_id': 'chunk 3', 'source': 'delivery_report.md'},
 {'chunk_id': 'chunk 4', 'source': 'delivery_report.md'},
 {'chunk_id': 'chunk 5', 'source': 'delivery_report.md'},
 {'chunk_id': 'chunk 6', 'source': 'delivery_report.md'},
 {'chunk_id': 'chunk 7', 'source': 'delivery_report.md'},
 {'chunk_id': 'chunk 8', 'source': 'delivery_report.md'},
 {'chunk_id': 'chunk 9', 'source': 'delivery_report.md'},
 {'chunk_id': 'chunk 10', 'source': 'delivery_report.md'},
 {'chunk_id': 'chunk 11', 'source': 'delivery_report.md'},
 {'chunk_id': 'chunk 12', 'source': 'delivery_report.md'},
 {'chunk_id': 'chunk 13', 'source': 'delivery_report.md'},
 {'chunk_id': 'chunk 14', 'source': 'delivery_report.md'},
 {'chunk_id': 'chunk 15', 'source': 'delivery_report.md'},
 {'chunk_id': 'chunk 16', 'source': 'delivery_report.md'}]

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import chromadb

# # # 1. Load docs
# # documents = SimpleDirectoryReader("./data").load_data()

# chunks

# 2. Setup vector store
chroma_client = chromadb.Client()
vector_store = ChromaVectorStore(chroma_collection=chroma_client.get_or_create_collection("my_docs"))

# 3. Create index
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
index = VectorStoreIndex.from_documents(
    documents=documents,
    metadatas=metadatas,
    ids=ids,
    embed_model=embed_model,
    vector_store=vector_store)


In [75]:
retriever = index.as_retriever(similarity_top_k=10)
results = retriever.retrieve("Security Deposit?")
for node in results:
    print("success------------------------------------------------------------------------")
    print(node.get_content())

success------------------------------------------------------------------------
## 11 Insurance; Waiver of Subrogation\_

(a) Insurance Digital   Realty and Customer will each maintain; and Customer will cause its contractors and others having access to the Customer Area to maintain the following insurance:

| Commercial General Liability (CGL) insurance or local equivalent                                                                                                                             | $1,000,000 USD per occurrence or equivalent sum in local currency   |
|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------|
| Statutory Workers' Compensation and Employers Liability insurance or local equivalent                                                                             

## Reranker

In [77]:
from llama_index.core.schema import NodeWithScore, QueryBundle, TextNode
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

# Assume 'results' is a list of nodes from your retriever
# Example: results = retriever.retrieve("your query")

# 1. Create reranker
reranker = FlagEmbeddingReranker(
    top_n=5,  # Number of top reranked results you want
    model="BAAI/bge-reranker-large",
    use_fp16=False
)

# 2. Wrap results as NodeWithScore objects
nodes = [NodeWithScore(node=TextNode(text=node.get_content())) for node in results]

# 3. Prepare query bundle
query = "Security Deposit?"
query_bundle = QueryBundle(query_str=query)

# 4. Rerank nodes
ranked_nodes = reranker._postprocess_nodes(nodes, query_bundle)

# 5. Print reranked content and scores
for node in ranked_nodes:
    print(node.node.get_content())
    print("Score:", node.score)
    print("-" * 120)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


## 5. Fees; Taxes and Other Charges; Security Deposit.

(a) Recurring Fees Starting on applicable Commencement Date, Customer shall pay in advance on the first day of each calendar month during the applicable Term; the monthly recurring charges or fees ( MRCs") per the applicable Order. If the Commencement Date is not on the first day of a calendar month the payment of the applicable MRCs for that month   will be pro-rated based on daily   rate calculated by dividing the MRCs by the total number of calendar days in that month: Unless otherwise expressed in an Order, during the Term; including any Extension Term; the MRCs shall increase by 3% annually from the prior year's MRCs on or after each anniversary of the Commencement Date the

(b) Other Fees. Customer shall pay in accordance with the applicable Order or invoice any non-recurring   charges ("NRCs") installationlde-installation fees; and technical support fees Unless otherwise stated in the applicable Order or invoice Customer mu

### llm Refinement

##### However, in llama_index, when you call index.as_query_engine() and then .query("..."), the default behavior is to use an LLM to generate a natural language answer from the retrieved chunks.

In [ ]:
# 4. Query
query_engine = index.as_query_engine()
response = query_engine.query("")
print(response)


In [ ]:
# After reranking
reranked_texts = [node.node.get_content() for node in ranked_nodes]

# Combine reranked texts for LLM refinement
context = "\n\n".join(reranked_texts)

# Use llama-index's LLM integration (Groq example)
from llama_index.llms.groq import Groq

llm = Groq(api_key="", model="llama3-70b-8192")  # Use your Groq model

# Ask the LLM to synthesize/refine the answer
prompt = f"Based on the following context, answer the query: Security Deposit?\n\n{context}"

response = llm.complete(prompt)
print(response)

The section related to Security Deposit is:

**(f) Security Deposit.** Digital Realty may require Customer to deposit at any time a security deposit in an amount equal to one month of MRCs or other amount as set forth in the first Order placed with respect to the Building. If Customer's required monthly payments are not received within 5 days after they are due for 2 consecutive months, then Digital Realty may, in its sole discretion, increase the required security deposit; and within 3 business days after receipt of written notice thereof, deposit with Digital Realty such sums as may be required to increase the security deposit to its required increased amount. Customer agrees that any security deposit provided may be applied against any Order that incorporates the terms of its applicable Country MSA and be used as security for the performance of Customer's obligations or liabilities under any such Order. The security deposit is not an advance payment of fees; and Digital Realty will 

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-base-en-v1.5")
tokens = tokenizer.encode("Hello world!")
print(len(tokens))  # [101, 7592, 2088, 999, 102]


In [ ]:
from typing import List
import nltk
from transformers import AutoTokenizer

nltk.download('punkt')  # For sentence tokenization

def chunk_text_for_embedding(
    text: str,
    model_name: str = "BAAI/bge-base-en-v1.5",
    max_tokens: int = 512,
    buffer_tokens: int = 20
) -> List[str]:
    """
    Splits input text into chunks that fit within the embedding model's token limit.

    Args:
        text: The input text.
        model_name: Name of the HuggingFace model to load the tokenizer.
        max_tokens: Max tokens allowed per chunk.
        buffer_tokens: Safety margin for tokenizer variations.

    Returns:
        List of text chunks, each within token limits.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    sentences = nltk.sent_tokenize(text)
    
    chunks = []
    current_chunk = []
    current_tokens = 0

    for sentence in sentences:
        token_count = len(tokenizer.encode(sentence, add_special_tokens=False))
        if current_tokens + token_count <= max_tokens - buffer_tokens:
            current_chunk.append(sentence)
            current_tokens += token_count
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = [sentence]
            current_tokens = token_count

    # Append last chunk
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    
    return chunks
